# Modelado de los residuos restantes de los cristales


### Obtener la secuencia de UniProt

In [15]:
# Librerías
from modeller import * # licencia modeller necesaria
from prody import *
import csv
from modeller import * # licencia modeller necesaria
from modeller.automodel import *
from prody import * 
from Bio import pairwise2
import os
import glob

In [2]:

# Secuencia de la CDK2 de UniProt
cdk2_P24941 = "MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNHPNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHSHRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYYSTAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSFPKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL"

### Función find gaps

In [3]:
# Versión marzo 2018 *Anotar las modificaciones en esta versión y comentar lo pasado
def find_gaps(seq, r=1): # r es el número de residuos móviles al lado de la ventana del gap
    import re
    '''Encontrar el número e inicio y final de los gaps en el alineamiento'''
    seq_len = len(seq)
    gaps = list(re.finditer('[-]+', seq)) # todas las subsecuencias que tienen uno o más guiones
    num_gaps = len(gaps); gap_lengths = []
    gap_list = []; gap_window = []
    # Obtener el índice de inicio y final del gap
    for i , gap in enumerate(gaps, 1):
        start = gap.start() + 1
        end = gap.end()
        gap_lengths.append(end - start + 1)
        gap_list.append([start, end])
        end_right = end if end >= seq_len else end  + r
        start_right = start if start == 1 else start - r
        gap_window.append([start_right, end_right])
    return({"num_gaps": num_gaps, "gap_lengths":gap_lengths,
            "gap_list": gap_list, "gap_window": gap_window})

In [8]:
# EJAMPLO:
find_gaps("LLLL---LLLÑÑÑR-K--L", r=0)

{'num_gaps': 3,
 'gap_lengths': [3, 1, 2],
 'gap_list': [[5, 7], [15, 15], [17, 18]],
 'gap_window': [[5, 7], [15, 15], [17, 18]]}

### Obtener la secuencia del PDB del cristal

In [10]:
# Los IDs de los cristales están en el archivo
with open('./CDK2_pdb_391_IDs_P24941.csv', 'r') as f:
    reader = csv.reader(f)
    cdk2_pdb_ids = list(reader)[0] # Lista de IDs
len(cdk2_pdb_ids)

391

## Se ejecuta Modeller para refinamiento por loops

In [23]:
import os
import time
start = time.time()

# Directiros
# Mismo para todos
struct_dir = './PDB_CDK2_chains/'
tail_pdb = '_A.pdb' # extensión del archivo pdb del cristal
tail_model = '_full' # epiteto del archivo del modelo final, la extensión .pdb la agrega modeller
pdbs_model_dir = "./Modelos_Full_CDK2_PDB/" # Carpeta de salida para los modelos
num_res_window = 2

# *****************************************************************************
# Para corregir los modelos que salieron mal en la primera ejecución de modeller
# # Conformaciones que necesitan atención:
# /home/joel/Documentos/Doctorado/DOCTO_TESIS/Proteinas_Modelo/CDK2/CONFORMACIONES/Prueba_Modeller/Modelos_malos_primera iteración
# URGENTE: 1h00, 1oit, 1p2a, 2r3j, 2r3k, 4acm, 4ek6, 4fkj, 5ano, 6ath
# LOOPS muy extendidos: 1w98, 5l2w, 5uq3
# PRIMERA RONDA:
# Se intenta ampliando la ventana de 1 a 2 residuos
#cdk2_pdb_ids = ['1h00', '1oit', '1p2a', '2r3j', '2r3k', '4acm', '4ek6', '4fkj', '5ano', '6ath',
    #           '1w98', '5l2w', '5uq3']
# Originales: 1h00 1oit 1p2a 2r3j 2r3k 4acm 4ek6 4fkj 5ano 6ath 1w98 5l2w 5uq3

# SEGUNDA RONDA: (Los que no quedaron en la primera)
# 2r3k* 4acm* 4ek6* 6ath* 5uq3*
# Más exhaustividad a 1000, residuos a 0

# TERCERA RONDA: (Los que no quedaron en la segunda)
# 4ek6*: loops entrelazados 
# 6ath*: N terminal desplegada
# 5uq3*: dominio pequeño (N-terminal) sin estructura (critico)
# COCLUSIÓN: Descratar los tres debido a que les faltan hasta 16 residuos en la región N-terminal

# CUARTA RONDA: (Los que no funcionaban con AD4)
# 3s2p, 1oit, 1v1k
# window = 2, a.max_var_iterations = 2000 #*500, a.md_level = refine.slow # Nivel del rifinamiento,
# a.repeat_optimization = 3 ##Originalmente comentado

# QUINTA RONDA: (Los que cuyos loops solapaban con el ligando: altas energias en AD4)
# 4bcq, 3pxf, 2wip, 1oiq
# window = 1

cdk2_pdb_ids = ['2wip'] # No funcionó el refinamiento por loops

for pdb_code in cdk2_pdb_ids:
    #INPUTS
    # variable
    code = pdb_code
    
    ###########################
    if os.path.isfile(pdbs_model_dir + code + tail_model + ".pdb"):
        print("El modelo ya existe en el directorio")
        continue

    ###########################

    # Secuencia de la estructura cristalográfica
    try:
        stc_prot = parsePDB(struct_dir + code + tail_pdb).select('not nonstdaa and resid > 0') # Archivo pdb
                                                            # La selección omite residuos negativos
            # Además omite los residuos no estandar, con modificaciones postTrad como el TPO (Tirosina fosfatada)
        chaid = stc_prot.getChids()[0] # Obtiene el id de la cadena
        ref_hv = stc_prot.getHierView()[chaid] # Se obtiene sólo la cadena
        seq_cry = ref_hv.getSequence() # Se obtiene la secuencia
        
        #print(seq_cry)
        #print(cdk2_P24941)
       
    except:
        print("Error al abrir y modelar:", pdb_code)
        continue

    ########################### CONTROL
    # Pregunta si la longitud y la secuencias de Uniprot y de la estructura son iguales
    same_seq = len(seq_cry) == len(cdk2_P24941) and seq_cry == cdk2_P24941
    # si same_seq es verdadero, se omite hacer el modelado pues la estructura está completa
    # y se pasa a la siguiente secuencia
    
    if same_seq:
        print("La proteína " + code + " ya está completa")
        # continue
        
    print("Modelando proteína " + code)

    ###########################
    # Obtener la secuecnia de la estructura a modelar y guardarla en un archivo

    # Alineamiento global, se penalizan con -10 los gaps abiertos, se obtiene el mejor
    alignment = pairwise2.align.globalxs(seq_cry, cdk2_P24941, 
                                        -10, -0.5, gap_char='-')[0]
    # Secuencias del a
    algn1_struc = alignment[0] # Secuencia alineada de la estructura cristalográfica
    algn2_seq = cdk2_P24941 # alignment[1] == Secuencia completa de UniProt
    
    # print(algn1_struc)
    # print(algn2_seq)

    # Nombre de los cabezales de las secuencia
    # deben coincidir con el de los archivos de entrada y salida
    crys_file_name = code + tail_pdb
    model_file_name = code + tail_model

    # Necesario: There should be 10 fields separated by colons ":"
    # Please check the file to make sure your sequences end with the '*' character.
    # Nomenclaturas de los campos del header: https://salilab.org/modeller/8v2/manual/node176.html

    # Headers (la secuencia del cristal va primero, luego la secuencia completa)
    struc_header = "structureX:" + crys_file_name + ":.:" + chaid + ":.:" + chaid + ":.:.:.:"
    seq_header = "sequence:" + model_file_name + ":.:.:.:.:.:.:.:"

    # Crea el archivo de alineamiento con la estructura requerida por modeller
    alg_filename = code + ".alg"
    with open(alg_filename, "w") as handle:
        handle.write("\n>P1;%s\n%s\n%s*\n>P1;%s\n%s\n%s*\n" % (crys_file_name, struc_header, algn1_struc, 
                                                               model_file_name, seq_header, algn2_seq))

    ##########################
    gaps = find_gaps(algn1_struc, r = num_res_window)
    num_gaps = gaps["num_gaps"]
    print(num_gaps)
    
    ##### Comprueba si hay gaps
    if num_gaps != 0:
     #   print("Se omite hacer el modelado de " + code + ", la proteína ya está completa o tiene más residuos que la secuencia")
     #   print("Longitud de secuencia: " + str(len(algn2_seq)))
     #   print("Longitud de la estructura: " + str(len(algn1_struc)))
     #   continue

        gap_i = gaps["gap_window"]
        s = "self.residue_range('" + str(gap_i[0][0]) + "', '" + str(gap_i[0][1]) + "')"

        # A pesar de que sólo haya una, se ejecuta el ciclo, si hay más de una, se crean para las demás
        for i in range(1, num_gaps):
            s = s + ", " + "self.residue_range('" + str(gap_i[i][0]) + "', '" + str(gap_i[i][1]) + "')" #nótese la coma
        print(s)
        ##########################
        env = environ()
        env.io.atom_files_directory = ['.', struct_dir]

        # Modifico loopmodel para fijar los residuos ya existentes en el cristal
        # NECESARIO para poder capturar los valores obtenidos por el rango de gaps
        # Al estar dentro del loop la identación extra es importante
        MyModel_code = """
class MyModel(automodel):
    def select_atoms(self):
        return selection(""" + s + """)
    """

        exec(MyModel_code) # Se modifica la clase MyModel para agregar la región fijada

    a = MyModel(env, alnfile = alg_filename, # Lee el archivo fasta creado y guardado en el directorio actual
                      knowns = crys_file_name, # Archivo pdb crys, que coincide con el id en el archivo fasta
                      sequence = model_file_name) # Nombre del modelo
    a.starting_model= 1
    a.ending_model  = 1
    #a.library_schedule = autosched.slow ##Originalmente comentado
    a.max_var_iterations = 2000 #*500
    a.md_level = refine.slow # Nivel del rifinamiento
    a.repeat_optimization = 3 ##Originalmente comentado
    
    # Más info sobre el refinamiento: https://salilab.org/modeller/9.21/manual/node19.html
    a.make()

    ###########################
    # Renombra el archivo pdb a code + _full.pdb
    # DEBE ser el único archivo pdb en el directorio '.'

    model_file = glob.glob('./' + code + '*.pdb')[0] # Nombre del archivo pdb
    os.rename(model_file, pdbs_model_dir + code + "_full.pdb") # Mueve el pdb a la carpeta model_pdbs

    # Elimina los archivos extra generados - Todos empiezan con el codigo del PDB ID
    for f in glob.glob(code + "*"):
        os.remove(f)

end = time. time()
print(end - start)

@> 2391 atoms and 1 coordinate set(s) were parsed in 0.02s.


Modelando proteína 2wip
1
self.residue_range('9', '17')

check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.
              Total number of restraints before, now:    31997    29808
64 (of 2398 total) atoms selected for optimization
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle t

In [5]:
## Corrección de modelos

# PENDIENTE: Modeller por pasos, por comentar

## Gaps, covertura e identidad 
#### Para los metadatos de los cristales obtengo una tabla con el número de gaps, la covertura de la secuencia y su identidad.


### Hacer el alineamiento y crear el archivo de entrada para modeller
Se lleva a cabo un alineamiento global entre la secuencia de UniProt (la cual está completa) y la secuencia de la estructura del cristal, la cual puede tener gaps. El alineamiento es global y tiene una alta penalización  a la apertura de nuevos gaps, lo cual reduce el número posible de gaps concordando con las estructuras cristalográficas.

In [60]:
# Alineamiento
from Bio import pairwise2

# Primer alineamiento global, se penalizan con -10 los gaps abiertos
alignment = pairwise2.align.localxs(seq_cry, cdk2_P24941, -10, -0.5, 
                                    gap_char='-')[0] 
algn1_struc = alignment[0] # Estructura
algn2_seq = alignment[1]   # Secuencia UniProt

seq_id = "P24941"
cry_code = "4FKW"

# Necesario: There should be 10 fields separated by colons, :
# Please check the file to make sure your sequences end with the '*' character.
# Nomenclaturas de los campos del header: https://salilab.org/modeller/8v2/manual/node176.html

# Headers
struc_header = "structureX:" + cry_code + ":.:" + chaid + ":.:" + chaid + ":.:.:.:"
seq_header = "sequence:.:.:.:.:.:.:.:.:"

# PDB name / nombre del archivo que tendrá el pdb al final del modelado
tail_pdb = ".pdb_chA.pdb"

# Crea el archivo de alineamiento con la estructura requerida por modeller
filename = "alignment.fasta"
with open(filename, "w") as handle:
    handle.write("\n>P1;%s%s\n%s\n%s*\n>P1;%s\n%s\n%s*\n" % (cry_code, tail_pdb, struc_header, algn1_struc, 
                                                           seq_id, seq_header, algn2_seq))

### Función para identificar los gaps
Los gap se identifican en la secuencia de la estructura cristalográfica.

In [51]:
### Encontrar el número e inicio y final de los gaps en el alineamiento
import re

def find_gaps(seq, r=1): # r es el número de residuos móviles al lado de la ventana del gap
    gaps = re.findall('[-]+', seq) # todas las subsecuencias que tienen uno o más guiones
    num_gaps = len(gaps); gap_lengths = []
    gap_list = []; gap_window = []
    # Obtener el índice de inicio y final del gap
    for i , gap in enumerate(gaps,1):
        start = seq.index(gap) + 1
        end = start + len(gap) - 1
        gap_lengths.append(len(gap))
        gap_list.append([start, end])
        gap_window.append([start - r, end + r])
    return({"num_gaps": num_gaps, "gap_lengths":gap_lengths,
            "gap_list": gap_list, "gap_window": gap_window})

find_gaps(algn1_struc)

{'num_gaps': 2,
 'gap_lengths': [5, 13],
 'gap_list': [[38, 42], [150, 162]],
 'gap_window': [[37, 43], [149, 163]]}

In [56]:
# Preguntar si hay al menos un gap
# si sí, crear para el primero

gaps = find_gaps(algn1_struc)
num_gaps = gaps["num_gaps"]
gap_i = gaps["gap_list"]
s = "self.residue_range('" + str(gap_i[0][0]) + "', '" + str(gap_i[0][1]) + "')"

# A pesar de que sólo haya una, se ejecuta el ciclo, si hay más de una, se crean para las demás
for i in range(1, num_gaps):
    s = s + ", " + "self.residue_range('" + str(gap_i[i][0]) + "', '" + str(gap_i[i][1]) + "')" #nótese la coma
    
print(s)
    
# k = [exec("%s('37', '43')" % (x))]

self.residue_range('38', '42'), self.residue_range('150', '162')


In [85]:
from modeller import *
from modeller.automodel import *

# log.verbose()
env = environ()

env.io.atom_files_directory = ['.', '../PDBs_chains/']


# Modifico loopmodel para fijar los residuos ya existentes en el cristal
# NECESARIO para poder capturar los valores obtenidos por el rango de gaps
code = """
class MyModel(automodel):
    def select_atoms(self):
        return selection(""" + s + """)
"""

exec(code)

a = MyModel(env, alnfile = 'alignment.fasta',
              knowns = "4FKW.pdb_chA.pdb", sequence = seq_id)

a.starting_model= 1
a.ending_model  = 1


a.make()


check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.
              Total number of restraints before, now:    30219    28030
148 (of 2398 total) atoms selected for optimization
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_397W> Atoms were not swapped because